# CS581: Ridesharing Project
## Team 5

The following code handles preprocessing rides, filtering them, generating the ridesharing graph, and running the map-matching algorithms.

Please find required packages in `requirements.txt`. You can install them via `pip install -r requirements.txt`.

## Imports

In [39]:
import math
import time
from datetime import datetime, timedelta
from multiprocessing import Pool, cpu_count
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook
from h3 import h3
import copy

%matplotlib inline
np.random.seed = 1337

## Load rides and distances

- Load valid source and destination hexagons
- Load rides with relevant columns
- Convert pickup/dropoff times to datetime objects
- Filter rides by specifying pickup dates and months

In [40]:
%%time
file_path = "2016-06.csv" # PATH TO CSV FILE FROM NYC TAXI DATASET
s2s = pd.read_csv('data/db/s2s.csv').pickup_h3.unique()
d2d = pd.read_csv('data/db/d2d.csv').dropoff_h3.unique()
cols_to_keep = ['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'trip_distance', 'pickup_longitude',
       'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude', 'fare_amount', 'total_amount', 'RatecodeID']

d = pd.read_csv('data/nyc_taxi/' + file_path, usecols=cols_to_keep)
d.columns = ['pickup_datetime', 'dropoff_datetime', 'trip_distance', 'pickup_longitude', 
            'pickup_latitude', 'rate_code', 'dropoff_longitude', 'dropoff_latitude', 'fare_amount', 
            'total_amount']
d['pickup_datetime'] = pd.to_datetime(d['pickup_datetime'])
d['dropoff_datetime'] = pd.to_datetime(d['dropoff_datetime'])
date, month = 10, 6
d = d[(d['pickup_datetime'].dt.day == date) & (d['pickup_datetime'].dt.month == month)]
print(d.shape)

(401458, 10)
CPU times: user 36.3 s, sys: 3.79 s, total: 40.1 s
Wall time: 40.1 s


## Specify time if time constraints given
- Given a specific day, filter by the start and end times

In [41]:
start_date = '2016-06-10 08:00:00' # Start date with time
end_date = '2016-06-10 23:59:59' # End date with time
mask = (d['pickup_datetime'] >= start_date) & (d['dropoff_datetime'] <= end_date)
d_day = d[mask].reset_index().drop('index', axis=1)
print(d_day.shape)

(318665, 10)


## Convert (lat, long) to H3 hexagon identifiers and filter by valid hexagons
- Snap pickup and dropoff latitude, longitude pairs to nearest hexagon
- Filter them by valid hexagons

In [42]:
%%time
d_day['pickup_h3'] = d_day.apply(lambda x: h3.geo_to_h3(x['pickup_latitude'], x['pickup_longitude'], 8), axis=1)
d_day['dropoff_h3'] = d_day.apply(lambda x: h3.geo_to_h3(x['dropoff_latitude'], x['dropoff_longitude'], 10), axis=1)
d_day = d_day[d_day['pickup_h3'].isin(s2s)].reset_index()
d_day = d_day[d_day['dropoff_h3'].isin(d2d)].reset_index()
d_day = d_day.drop(['index', 'level_0'], axis=1)
print(d_day.shape)

(3585, 12)
CPU times: user 22.5 s, sys: 415 ms, total: 23 s
Wall time: 22.9 s


## Generate pool window times
- Ceil pickup times to the nearest pool window time for generating pool groups

In [43]:
def ceil_dt(dt, delta):
    return datetime.min + math.ceil((dt - datetime.min) / delta) * delta

pool_time_window = 10 # Change pool time window
d_day['pool_window'] = d_day['pickup_datetime'].apply(lambda x: ceil_dt(x.to_pydatetime(), timedelta(minutes=pool_time_window)))
d_day['duration'] = (d_day['dropoff_datetime'] - d_day['pickup_datetime']).dt.seconds
d_day['delay'] = d_day['duration'].apply(lambda x: x*0.20)

## Load precomputed distances and durations
- Load distances and durations for source-to-source, source-to-destination, and destination-to-destination

In [44]:
%%time
s2s_path = 'data_h3/db/s2s.csv'
s2d_path = 'data_h3/db/s2d.csv'
d2d_path = 'data_h3/db/d2d.csv'

s2s = pd.read_csv(s2s_path)
s2d = pd.read_csv(s2d_path)
d2d = pd.read_csv(d2d_path)
org_dist = pd.concat([s2s, s2d, d2d])
dist = org_dist.set_index(['pickup_h3', 'dropoff_h3'])
del s2s, s2d, d2d
print(dist.shape)

(13764208, 2)
CPU times: user 10.1 s, sys: 2.97 s, total: 13.1 s
Wall time: 13.1 s


## Node Definition and Creation
- Specify social affinity matrices
- Create custom Node classes with required fields
- Function for generating all possible ride combinations
- Function to calculate edge weights
- Sort the graphs by the largest size first (optimization technique for multiprocessing)

In [45]:
%%time

professions = ['musician', 'engineer', 'doctor', 'lawyer', 'actor']
languages = ['english', 'french', 'spanish', 'hindi']
aff_prof = [
    [1, 0.4, 0.5, 0.1, 0.85], 
    [0.4, 1, 0.8, 0.4, 0.3], 
    [0.5, 0.8, 1, 0.6, 0.2], 
    [0.1, 0.4, 0.6, 1, 0.7], 
    [0.85, 0.3, 0.2, 0.7, 1]
]
aff_lang = [
    [1, 0.6, 0.78, 0.7],
    [0.6, 1, 0.7, 0.2], 
    [0.78, 0.7, 1, 0.3],
    [0.7, 0.2, 0.3, 1]
]

class Node:
    def __init__(self, data, id):
        self.id = id
        self.pickup_time = data.pickup_datetime
        self.dropoff_time = data.dropoff_datetime
        self.pickup_loc = (data.pickup_longitude, data.pickup_latitude)
        self.dropoff_loc = (data.dropoff_longitude, data.dropoff_latitude)
        self.amt = data.fare_amount
        self.total_amt = data.total_amount
        self.pickup_h3 = data.pickup_h3
        self.dropoff_h3 = data.dropoff_h3
        self.delay = data.delay
        self.trip_dist = dist.loc[(data.pickup_h3, data.dropoff_h3), ['distance']]['distance']
        self.duration = dist.loc[(data.pickup_h3, data.dropoff_h3), ['duration']]['duration']
        self.profession = np.random.choice(professions)
        self.language = np.random.choice(languages)
        
def generate(a, b):
    pa_pb, dur_pa_pb = dist.loc[(a.pickup_h3, b.pickup_h3), ['distance', 'duration']]
    pb_pa, dur_pb_pa = dist.loc[(b.pickup_h3, a.pickup_h3), ['distance', 'duration']]
    da_db, dur_da_db = dist.loc[(a.dropoff_h3, b.dropoff_h3), ['distance', 'duration']]
    db_da, dur_db_da = dist.loc[(b.dropoff_h3, a.dropoff_h3), ['distance', 'duration']]
    
    pb_da, dur_pb_da = dist.loc[(b.pickup_h3, a.dropoff_h3), ['distance', 'duration']]
    pb_db, dur_pb_db = dist.loc[(b.pickup_h3, b.dropoff_h3), ['distance', 'duration']]
    pa_db, dur_pa_db = dist.loc[(a.pickup_h3, b.dropoff_h3), ['distance', 'duration']]
    pa_da, dur_pa_da = dist.loc[(a.pickup_h3, a.dropoff_h3), ['distance', 'duration']]
    
    i = pa_pb + pb_da + da_db
    j = pa_pb + pb_db + db_da
    k = pb_pa + pa_db + db_da
    l = pb_pa + pa_da + da_db
    
    d_i = dur_pa_pb + dur_pb_da + dur_da_db
    d_ia, d_ib = dur_pa_pb + dur_pb_da, d_i
    
    d_j = dur_pa_pb + dur_pb_db + dur_db_da
    d_ja, d_jb = d_j, dur_pa_pb + dur_pb_db
    
    d_k = dur_pb_pa + dur_pa_db + dur_db_da
    d_ka, d_kb = d_k, dur_pb_pa + dur_pa_db
    
    d_l = dur_pb_pa + dur_pa_da + dur_da_db
    d_la, d_lb = dur_pb_pa + dur_pa_da, d_l
    return [(i, d_i, d_ia, d_ib), (j, d_j, d_ja, d_jb), (k, d_k, d_ka, d_kb), (l, d_l, d_la, d_lb)]
    
graphs = []
for time, df in tqdm_notebook(d_day.groupby(['pool_window']), 
                              total=len(d_day.groupby(['pool_window']))):
    nodes = []
    df = df.reset_index()
    for idx, row in df.iterrows():
        nodes.append(Node(df.iloc[idx], idx))
    G = nx.Graph()
    G.add_nodes_from(nodes)
    graphs.append(G)
    
def target_func(gr):
    def edge_weight_calc(a, b, w1=0.25, w2=0.75, w3=0.80):
            t_min = float('inf')
            d_min = float('inf')
            R = generate(a, b)
            t_a = a.trip_dist
            t_b = b.trip_dist
            d_a = a.duration
            d_b = b.duration
            for (t, d, dp_a, dp_b) in R:
                if t <= (t_a + t_b) and t <= t_min and dp_a <= (d_a + a.delay) and dp_b <= (d_b + b.delay) and d <= d_min:
                    t_min = t
                    d_min = d
            pf = aff_prof[professions.index(a.profession)][professions.index(b.profession)]
            la = aff_lang[languages.index(a.language)][languages.index(b.language)]
            
            return [w1*(t_a + t_b - t_min), w2*(d_a + d_b - d_min), w3*(pf+la)/2]
    for a in gr:
        for b in gr:
            if a.id == b.id: continue
            [t_w, d_w, soc] = edge_weight_calc(a, b)
            if t_w != float('-inf') and d_w != float('-inf'):
                gr.add_edge(a, b, weight={'distance': t_w, 'duration': d_w, 'final': soc*(t_w+d_w)})
    return gr

graphs = sorted(graphs, key=lambda x: x.number_of_nodes(), reverse=True)


CPU times: user 9.52 s, sys: 444 ms, total: 9.97 s
Wall time: 9.8 s


## Edge weight calculation
- Run edge weight calculation in parallel scaled by the number of available physical CPUs
(This may take a while depending on the number of available CPUs)

In [46]:
%%time
p = Pool(processes=cpu_count())
data = p.map(target_func, graphs)
p.close()

CPU times: user 2.1 s, sys: 2.23 s, total: 4.33 s
Wall time: 1min 37s


## Max Weighted Matching Algorithm
- Run `max_weight_matching` algorithm on all our pool graphs

In [48]:
weight_matches = []
for graph in tqdm_notebook(data, total=len(graphs)):
    match = nx.max_weight_matching(graph, weight='final', maxcardinality=True)
    g_match = nx.Graph()
    for ii in match:
        g_match.add_edge(ii[0],ii[1])
    weight_matches.append(g_match)

## Utilization
- Calculate the utilization for each pool graph and average them

In [50]:
org_nodes = [x.number_of_nodes() for x in graphs]
weight_match_nodes = [x.number_of_nodes() for x in weight_matches]

print(sum([(y/x) for x,y in zip(org_nodes, weight_match_nodes)]) / len(graphs)*100)

95.17672320116023
97.28280247939068


## Average distance saved per pool as a % of total distance of individual rides
- Calculate the average distance saved per pool as a percentage of total distance of individual rides

In [51]:
new_dist, org_dist = [], []

for idx in tqdm_notebook(range(len(weight_matches)), total=len(weight_matches)):
    org_d, d = 0, 0
    individual_nodes = set()
    for node in data[idx].nodes:
        org_d += node.trip_dist
        individual_nodes.add(node)
    for edge in weight_matches[idx].edges:
        individual_nodes.remove(edge[0])
        individual_nodes.remove(edge[1])
        d += data[idx].get_edge_data(edge[0], edge[1])['weight']['distance']*4
    for node in individual_nodes:
        d += node.trip_dist
    new_dist.append(d)
    org_dist.append(org_d)

print(sum([(1-x/y) for x, y in zip(new_dist, org_dist)])/len(org_dist) * 100)


55.738847205366106


## Average number of trips saved per pool as a % of number of individual trips
- Calculate the average number of trips saved per pool as a percentage of number of individual rides

In [52]:
saved_rides = []
for idx in range(len(data)):
    num_ind_trips = len(data[idx].nodes)
    num_pooled_trips = len(weight_matches[idx].edges)
    saved_rides.append(num_pooled_trips/num_ind_trips * 100)
print(sum(saved_rides)/len(saved_rides))

48.64140123969537
